In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import re

spark = SparkSession.builder.config("spark.jars", "/Users/pietro-avro_2.12-3.1.3").getOrCreate()

In [19]:
# TEST THIS HAS TO BE DELETED!!!!!!!!!!!!!!
test=spark.read.csv("hdfs://localhost:9000/persistent/chile/metadata/imp/2022/21463104042022/EXPheaders.csv")

In [ ]:
hdfs_path = 'hdfs://localhost:9000/persistent/'

In [2]:
###
# How it works
###
# 1) metadata is opened
# 2) metadata is formatted as (code, name to be kept in the data)
# 3) data is formatted as (code for the join, all row)
# the variables that only have to have different names with the same values are kept as they are
# the metadata is treated as coming from txt files, not avros.
# Avro are always opened as set of Row objects.

In [2]:
# FUNCTIONS

def delete(row, col_name):
    d = row.asDict()
    d.pop(col_name)
    return Row(**d)

# function add one key to a row
def insert(row, new_col_name, value):
    d = row.asDict()
    d[new_col_name] = str(value).rstrip()
    return Row(**d)
    
def lenght(val):
    if isinstance(val, int):
        return 1
    else:
        return len(val)

# update one field in row
def updateRow(row, field_to_update, value):
    d = row.asDict()
    d[field_to_update] = value 
    return Row(**d)

def changeNames(row, old_names, new_names):
    for i in range(0, len(old_names)):
        row = insert(row, new_names[i], row[old_names[i]])
        row = delete(row, old_names[i])
    return row

def keepOnly(row, fields_to_be_kept):
    new_row = Row(var=True)
    for field in fields_to_be_kept:
        new_row = insert(new_row, field, row[field])
    new_row = delete(new_row, "var")
    return new_row  

def merging_data_meta (rdd, meta_file_name, key_pos_in_meta, name_pos_in_meta, key_name_in_data, keep_code, new_col_name, key_as_int, path_to_metadata, country):
    
    # METADATA
    if country == "peru":
        # if the first element starts with "C" or "t", it has to be dropped (it is the header)
        meta_split = sc.textFile(path_to_metadata + meta_file_name).filter(lambda l: not l.startswith(tuple(["C","t"]))).map(lambda l: tuple(l.split("\t")))
        #print(meta_split.take(4))
        #print(meta_split.take(2)) 
    
    elif country=="brasil":
        if meta_file_name == "NCM.csv":
            delim = ";"
            meta_split = sc.textFile(path_to_metadata + meta_file_name).filter(lambda l: not l.startswith("C")).map(lambda l: tuple(tuple(l.split("\n"))[0].split(delim)))
        else:
            delim="\",\""
            meta_split = sc.textFile(path_to_metadata + meta_file_name).filter(lambda l: not l.startswith("\"C")).map(lambda l: tuple(tuple(l.split("\n"))[0][1:-1].split(delim)))
    else:
        print('--- ERROR: wrong country name --')
    # prepare metadata for the join
    if lenght(key_pos_in_meta) == 2: # the key is the combination of 2 columns
        meta_ready_for_join0 = meta_split.map(lambda l: ( l[key_pos_in_meta[0]].strip() + l[key_pos_in_meta[1]].strip(), l[name_pos_in_meta].rstrip())).distinct()
    elif (key_as_int):
        meta_ready_for_join0 = meta_split.map(lambda l: ( int(l[key_pos_in_meta].strip()), l[name_pos_in_meta].rstrip())).distinct()
    else:
        meta_ready_for_join0 = meta_split.map(lambda l: ( l[key_pos_in_meta].strip(), l[name_pos_in_meta].rstrip())).distinct() 
    
    # if there are repetiotions of the key with typos on the attr. name that should instead be always the same
    meta_ready_for_join = meta_ready_for_join0.groupByKey().mapValues(list).map(lambda l: (l[0],l[1][0]))

    # DATA
    # prepare data for the join:
    if (key_as_int):
        # if there are nan, we set them to 9999999999
        data_ready_for_join = rdd.map(lambda l: updateRow(l, key_name_in_data, "9999999999") if l[key_name_in_data]=='nan' else l).map(lambda l: (int( l[key_name_in_data].split(".")[0]) , l) )
        # print(data_ready_for_join.take(5))
    else:
        data_ready_for_join = rdd.map(lambda l: ( l[key_name_in_data], l))
   
    # JOIN
    join = data_ready_for_join.leftOuterJoin(meta_ready_for_join)
    
    if (not keep_code): # remove the column with the code
        merged = join.map(lambda l:  (delete(l[1][0], key_name_in_data), l[1][1]) )
        #print(merged.take(1))

    else:
        merged = join.map(lambda l:  (l[1][0], l[1][1]) )
    
    complete = merged.map(lambda l:  insert(l[0], new_col_name, l[1]) )
    
    return complete



def create_composed_columns(rdd, source_col_names, dest_col_name, string = False, operation = "+"):
    
    # "+" (addition for numbers or concatenating strings)
    # "/" (division for numberes)
    
    if operation == "+":
        if string:
            return rdd.map(lambda l: insert(l, dest_col_name, ' '.join(l[s].strip() for s in source_col_names if l[s])))
        return rdd.map(lambda l: insert(l, dest_col_name, int(float(l[source_col_names[0]].replace(",","."))) + int(float(l[source_col_names[1]].replace(",","."))) ))
    if operation == "/":
        return rdd.map(lambda l: insert(l, dest_col_name, int(float(l[source_col_names[0]].strip())) / (int(float(l[source_col_names[1]].strip())) + 0.000001)))



In [3]:
def main_peru(path_to_avro=True, path_to_metadata=True):
    
    if path_to_avro:
        path_to_avro = '/Users/pietro/Desktop/version0.avro'
    if path_to_metadata:
        path_to_metadata = '/Users/pietro/Desktop/BDM/Project/Data.nosync/peru/metadata_copia/'
    df = spark.read.format('avro').load(path_to_avro)
    rdd=df.rdd
    
    
    # MERGING DATA AND METADATA
    meta_file_name = ["Partidas.txt", "Paises.txt", "Paises.txt", "Puertos.txt", "MedTransporte.txt", "Agente.txt", "Bancos.txt", "RecintAduaner.txt", "EstMercancia.txt"]
    key_pos_in_meta =  [0,0,0,[0,2],0,0,0,0,0]
    name_pos_in_meta = [1,1,1,  3,  1,1,1,1,1] # position of the columns in meta that contains the name we want to keep 
    key_name_in_data = ["PART_NANDI", "PAIS_ORIGE", "PAIS_ADQUI", "PUER_EMBAR", "VIA_TRANSP", "CODI_AGENT", "BANC_CANCE", "CODI_ALMA", "SEST_MERCA"]
    keep_code = [True, False, False, False, True, False, False, False, False] # true if we want to keep the code of the variable in the final data
    new_col_name = ["custom_description", "country_of_origin", "country_of_arrival", "port_of_boarding", "mean_of_transport", "agente", "bank", "warehouse", "state"]
    key_as_int = [True, False, False, False, True, True, True, False, False ]
    
    rdds = []
    rdds.append(rdd)
    n_merging = len(name_pos_in_meta)
    for i in range(0,n_merging):
        rdds.append(merging_data_meta(rdds[i], meta_file_name[i], key_pos_in_meta[i], name_pos_in_meta[i], key_name_in_data[i], keep_code[i], new_col_name[i], key_as_int[i], path_to_metadata, "peru")) 
        #rdds.append(rdd)
        
    # COMBINING COLUMNS
    source_col_names = [["FOB_DOLPOL",  "FLE_DOLAR"],["price_transport_net",  "SEG_DOLAR"], ["FOB_DOLPOL", "UNID_FIQTY"], ["DESC_COMER", "DESC_MATCO", "DESC_USOAP", "DESC_FOPRE", "DESC_OTROS"]]
    dest_col_name = [    "price_transport_net",     "price_transport_net_insurance",     "net_price_per_unit",          "commercial_description"]
    string = [False, False, False, True]
    operation = ["+", "+", "/", "+"]
    
    print("inizio il secondo")
    for i in range(0, len(dest_col_name)):
        rdds.append(create_composed_columns(rdds[n_merging + i], source_col_names[i], dest_col_name[i], string[i], operation[i])) 
    
    # CHANGING NAMES
    n_combining = len(source_col_names)
    
    old_names = ["CODI_ADUAN", "FECH_RECEP", "FOB_DOLPOL"]
    new_names = ["custom", "date", "net_price"]
    rdds.append(rdds[n_combining + n_merging].map(lambda l: changeNames(l, old_names, new_names)))
    
    
    # SELECT THE COLUMNS TO BE KEPT
    to_be_kept = ['country_of_arrival', 'mean_of_transport', 'price_transport_net', 'price_transport_net_insurance', 'net_price_per_unit', 'commercial_description', 'custom', 'date', 'net_price']
    rdds.append(rdds[n_combining + n_merging + 1].map(lambda l: keepOnly(l, to_be_kept)))

    # ADDING COUNTRY NAME and "IMP" label
    rdds.append(rdds[n_combining + n_merging + 2].map(lambda l: insert(l, "country", "peru")).map(lambda l: insert(l, "type", "IMP")))
    
    print(rdds[n_combining + n_merging + 3].take(1), "\n\n\n")
    return(rdds[n_combining + n_merging + 3])

In [4]:
def main_brazil(path_to_avro=True, path_to_metadata=True):
    
    if path_to_avro:
        path_to_avro = "/Users/pietro/Desktop/test_brazil.avro"
    if path_to_metadata:
        path_to_metadata = '/Users/pietro/Desktop/BDM/Project/Data.nosync/brazil/metadata copia/'
    df = spark.read.format('avro').load(path_to_avro)
    rdd=df.rdd
    
    # NOTE: NCM.csv must contain only the 2 columns of interest, with no \n in the strings
    meta_file_name = ["NCM.csv", "URF.csv", "PAIS.csv", "VIA.csv", "UF.csv"]
    key_pos_in_meta =  [0, 0, 0, 0, 1]
    name_pos_in_meta = [1, 1, 4, 1, 2] 
    key_name_in_data = ["CO_NCM", "CO_URF", "CO_PAIS", "CO_VIA", "SG_UF_NCM"]
    keep_code = [False, False, False, False, False] 
    new_col_name = ["custom_description", "custom", "country_of_arrival", "mean_of_transport", "arrival_place"]
    key_as_int = [True, True, True, True, False]
    
    rdds = []
    rdds.append(rdd)
    n_merging = len(name_pos_in_meta)
    for i in range(0,n_merging):
        rdds.append(merging_data_meta(rdds[i], meta_file_name[i], key_pos_in_meta[i], name_pos_in_meta[i], key_name_in_data[i], keep_code[i], new_col_name[i], key_as_int[i], path_to_metadata, "brasil")) 
        
    # COMBINING COLUMNS
    source_col_names = [["VL_FOB", "VL_FRETE"],["price_transport_net",  "VL_SEGURO"], ["VL_FOB", "KG_LIQUIDO"], ["CO_MES", "CO_ANO"]]
    dest_col_name = ["price_transport_net", "price_transport_net_insurance", "net_price_per_unit", "date"]
    string = [False, False, False, True]
    operation = ["+", "+", "/", "+"]
    
    for i in range(0, len(dest_col_name)):
        rdds.append(create_composed_columns(rdds[n_merging + i], source_col_names[i], dest_col_name[i], string[i], operation[i])) 
    
    # CHANGING NAMES
    n_combining = len(source_col_names)
    
    old_names = ["VL_FOB", "custom_description"]
    new_names = [ "net_price", "commercial_description"]
    rdds.append(rdds[n_combining + n_merging].map(lambda l: changeNames(l, old_names, new_names)))
    
    # SELECT THE COLUMNS TO BE KEPT
    to_be_kept = ['country_of_arrival', 'mean_of_transport', 'price_transport_net', 'price_transport_net_insurance', 'net_price_per_unit', 'commercial_description', 'custom', 'date', 'net_price']
    rdds.append(rdds[n_combining + n_merging + 1].map(lambda l: keepOnly(l, to_be_kept)))
     
    # ADDING COUNTRY NAME and "IMP" label
    rdds.append(rdds[n_combining + n_merging + 2].map(lambda l: insert(l, "country", "brazil")).map(lambda l: insert(l, "type", "IMP")))
    return(rdds[n_combining + n_merging + 3])
    


In [5]:
def main_chile(path_to_avro=True):
    
    if path_to_avro:
        path_to_avro = "/Users/pietro/Desktop/test_chile.avro"
    df = spark.read.format('avro').load(path_to_avro)
    rdd=df.rdd
        
    ### COLLAPSE
    source_col_names = [["FOB", "FLETE"],["price_transport_net",  "SEGURO"], ['DNOMBRE', 'DMARCA', 'DVARIEDAD', 'DOTRO1', 'DOTRO2', 'ATR_5',  'ATR_6']]
    dest_col_name = ["price_transport_net", "price_transport_net_insurance", "commercial_description"]
    string = [False, False, True]
    operation = ["+", "+", "+"]
    rdds = []
    rdds.append(rdd)
    for i in range(0, len(dest_col_name)):
        rdds.append(create_composed_columns(rdds[i], source_col_names[i], dest_col_name[i], string[i], operation[i])) 
        
    # CHANGING NAMES
    n_combining = len(source_col_names)
    old_names = ["ADU", "FECVENCI", "FOB", "PRE_UNIT", "CODPAISCON", "VIA_TRAN"]
    new_names = [ "custom", "date", "net_price", "net_price_per_unit", "country_of_arrival", "mean_of_transport"]
    rdds.append(rdds[n_combining].map(lambda l: changeNames(l, old_names, new_names)))
    
    # SELECT THE COLUMNS TO BE KEPT
    to_be_kept = ['country_of_arrival', 'mean_of_transport', 'price_transport_net', 'price_transport_net_insurance', 'net_price_per_unit', 'commercial_description', 'custom', 'date', 'net_price']
    rdds.append(rdds[n_combining + 1].map(lambda l: keepOnly(l, to_be_kept)))
    #print(rdds[n_combining  + 2].take(1), "\n\n\n")

    # ADDING COUNTRY NAME and "IMP" label
    rdds.append(rdds[n_combining + 2].map(lambda l: insert(l, "country", "chile")).map(lambda l: insert(l, "type", "IMP")))
    print(rdds[n_combining  + 3].take(1), "\n\n\n")
    return(rdds[n_combining  + 3]) 

In [6]:
# LOAD TO THE DB

data_chile=main_chile()
chileDF = data_chile.toDF()

data_brazil=main_brazil()
brazilDF = data_brazil.toDF()

data_peru=main_peru()
peruDF = data_peru.toDF()

properties = {"user": "pietro", "password": "password", "driver": 'org.postgresql.Driver'}
url = "jdbc:postgresql://localhost:5432/dataimporta"

chileDF.write.format("jdbc").mode("append").jdbc(url,"all_countries",
          properties = properties)
brazilDF.write.format("jdbc").mode("append").jdbc(url,"all_countries",
          properties = properties)
peruDF.write.format("jdbc").mode("append").jdbc(url,"all_countries",
          properties = properties)



inizio il secondo


[Row(country_of_arrival='UNITED STATES', mean_of_transport='AVION', price_transport_net='2294', price_transport_net_insurance='2316', net_price_per_unit='2199.9978000022', commercial_description='BOMBA DE COMBSUTIBLE, S/M, S/M EN UNIDAD/ INCLUYE ACCESORIOS PARA SU NORMAL FUNCIONAMIENTO PARA PROPORCIONAR COMBUSTIBLE AL MOTOR /ES PARA USO DE AERONAVE BOMBA DE COMBSUTIBLE METAL,P/N: 897400-7', custom='235', date='20220209', net_price='2200.0', country='peru', type='IMP')] 





In [10]:
# DATA FOR TBOX GRAPH
def tbox_peru(path_to_avro=True, path_to_metadata=True):
    
    if path_to_avro:
        path_to_avro = '/Users/pietro/Desktop/version0.avro'
    if path_to_metadata:
        path_to_metadata = '/Users/pietro/Desktop/BDM/Project/Data.nosync/peru/metadata_copia/'
    df = spark.read.format('avro').load(path_to_avro)
    rdd=df.rdd
    
    
    # MERGING DATA AND METADATA
    meta_file_name = ["Partidas.txt", "Paises.txt", "Paises.txt", "MedTransporte.txt"]
    key_pos_in_meta =  [0,0,0,0]
    name_pos_in_meta = [1,1,1,1] # position of the columns in meta that contains the name we want to keep 
    key_name_in_data = ["PART_NANDI", "PAIS_ORIGE", "PAIS_ADQUI", "VIA_TRANSP"]
    keep_code = [True, False, False, False] # true if we want to keep the code of the variable in the final data
    new_col_name = ["custom_description", "country_of_origin", "country_of_arrival", "mean_of_transport"]
    key_as_int = [True, False, False, True]
    
    rdds = []
    rdds.append(rdd)
    n_merging = len(name_pos_in_meta)
    
    for i in range(0,n_merging):
        rdds.append(merging_data_meta(rdds[i], meta_file_name[i], key_pos_in_meta[i], name_pos_in_meta[i], key_name_in_data[i], keep_code[i], new_col_name[i], key_as_int[i], path_to_metadata, "peru")) 
        #rdds.append(rdd)
   
    # CHANGING NAMES
    
    old_names = ["EMPR_TRANS", "DNOMBRE", "FOB_DOLPOL"]
    new_names = ["shipper", "trader", "net_price"]
    rdds.append(rdds[n_merging].map(lambda l: changeNames(l, old_names, new_names)))
    
    
    # SELECT THE COLUMNS TO BE KEPT
    to_be_kept = ["custom_description", "country_of_origin", "country_of_arrival", "mean_of_transport", "shipper", "trader", "net_price"]
    rdds.append(rdds[n_merging + 1].map(lambda l: keepOnly(l, to_be_kept)))
    return(rdds[n_merging + 2])
df = tbox_peru().toDF()

df.toPandas().to_csv('/Users/pietro/Desktop/peru_tbox.csv')